In [1]:
import pandas as pd

from df.keiba_data_frame import KeibaDataFrame
from df.result_frame import ResultFrame
from env.reward import Reward

# Train Data 読み込み
# ※ 既に AI 側で filter/data 整備は実施済みのため再 type 明確化と、特定列を Drop のみ行っている。
kdf = KeibaDataFrame.from_data("./../data/train/2013_train_binary.csv")
kdf.setting_types()

# Reward 準備
reward = Reward.initialize(["./../data/train/2013_result_wide.csv"])

In [2]:
import numpy as np
from tensorflow import keras

new_model = keras.models.load_model("./../model/20230406_binary_model_wide")
new_model.summary()

preds = []

i = 0
for v in list(kdf.groups()):
    v.reset_index(inplace=True, drop=True)
    #obs = v.reindex(range(20), fill_value=-1).values.reshape(1, 1, 20, 51).tolist()
    obs = v[["odds", "pred"]].reindex(range(20), fill_value=-1).values.reshape(1, 1, 20, 2).tolist()
    #print(obs)
    q_values = new_model.predict(obs)
    #print(q_values)
    action = np.argmax(q_values[0])
    #print(action)
    pred_df = pd.DataFrame(
        {
            "track_id": v["track_id"],
            "date_num": v["date_num"],
            "round": v["round"],
            "number": v["number"],
            "pred": v["pred"],
            "action": action,
        }
    )
    preds.append(pred_df)
    # i += 1
    # if i > 10:
    #     break

preds_df = pd.concat(preds)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 40)                0         
                                                                 
 dense (Dense)               (None, 128)               5248      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                        

In [3]:
from env.result_summary import ResultSummary
from env.actions import Action

result_summary = ResultSummary.initialize(Action.values())

df = preds_df.sort_values(
    ["date_num", "track_id", "round"], ascending=[True, True, True]
)

for _, v in df.groupby(["date_num", "track_id", "round"], sort=False):
    action = Action.of(int(v.iloc[0]["action"]))
    calc_reward = reward.calc(v, action)

    result_summary.add(action, calc_reward)

print(result_summary.to_result())


RANK_ONE_TWO_HORSE            :         -21500円 的中率 31.28%   回収率 88.97%   (61/195)
RANK_ONE_THREE_HORSE          :         -65600円 的中率 21.85%   回収率 56.56%   (33/151)
RANK_ONE_FOUR_HORSE           :         -42400円 的中率 21.62%   回収率 61.80%   (24/111)
RANK_ONE_FIVE_HORSE           :         -22800円 的中率 28.28%   回収率 76.97%   (28/99)
RANK_TWO_THREE_HORSE          :         -45200円 的中率 23.53%   回収率 70.46%   (36/153)
RANK_TWO_FOUR_HORSE           :          -2500円 的中率 27.78%   回収率 97.22%   (25/90)
RANK_TWO_FIVE_HORSE           :          -8100円 的中率 22.97%   回収率 89.05%   (17/74)
RANK_THREE_FOUR_HORSE         :         -29000円 的中率 16.67%   回収率 59.72%   (12/72)
RANK_THREE_FIVE_HORSE         :         -34300円 的中率 15.48%   回収率 59.17%   (13/84)
RANK_FOUR_FIVE_HORSE          :         -14300円 的中率 11.54%   回収率 45.00%   (3/26)
NO_ACITON                     :              0円 的中率 0.00%    回収率 0.00%    (0/2399)
TOTAL                         :        -285700円 的中率 23.89%   回収率 72.92%   (252/1055)
